In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify
import tensorflow as tf
import cv2
import os
import mysql.connector

# Initialize Flask application
app = Flask(__name__)

# Load the trained model
model = tf.keras.models.load_model('./py_model_classification.ipynb')

# Define route for image classification
@app.route('/classify', methods=['POST'])
def classify_image():
    # Get image file from request
    file = request.files['image']
    
    # Save image temporarily
    image_path = 'temp.jpg'
    file.save(image_path)
    
    # Preprocess image
    img = cv2.imread(image_path)
    resized_img = cv2.resize(img, (256, 256))
    resized_img = resized_img / 255.0  # Normalize
    
    # Make prediction
    prediction = model.predict(np.expand_dims(resized_img, axis=0))
    
    # Process prediction and send response
    class_names = ['Class1', 'Class2', 'NEWSPAPER', 'unknown']  # Define your class names
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_names[predicted_class_index]
    
    # Delete temporary image file
    os.remove(image_path)
    
    return jsonify({'prediction': predicted_class})

# Run Flask application
if __name__ == '__main__':
    app.run(debug=True)
